In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

In [3]:
indices = ['^GSPC', '^IXIC', '^GDAXI', '^HSI', '^N225']
equities = ['AAPL', 'AMZN', 'GOOGL', 'MSFT', 'TSLA']

In [11]:
start_date = "2010-01-01"
end_date = "2023-05-01"

data = yf.download(indices + equities, start=start_date, end=end_date)['Adj Close']

[*********************100%***********************]  10 of 10 completed


In [19]:
peak_value = data.max()
trough_value = data.min()
max_drawdowns = (trough_value - peak_value)/peak_value

In [30]:
'''
Risk free rates for 10 Year on May 1 2023
'''

US = 0.0353  #AAPL, AMZN, GOOGL, MSFT, TSlA, ^GSPC, ^IXIC
Germany = 0.0304  #^GDAXI
HKong = (0.0119*1 + 0.0426*3)/5 #^HSI (Linear interpolation between apr 28 and may 2)
Japan = 0.006    #^N225

In [31]:
Risk_free_rates = [US, US, US, US, US, Germany, US, HKong, US, Japan]

In [12]:
returns = data.pct_change()

In [13]:
cumulative_returns = (1 + returns).cumprod() - 1

In [14]:
rolling_max = cumulative_returns.rolling(window=len(cumulative_returns), min_periods=1).max()
drawdowns = cumulative_returns / rolling_max - 1


In [50]:
excess_returns = returns*255 - Risk_free_rates
annual = returns*255
sharpe_ratio = excess_returns.mean() / (returns.std() * np.sqrt(252))

In [51]:
downside_returns = np.where(returns < 0, returns, 0)
sortino_ratio = excess_returns.mean() / (downside_returns.std() * np.sqrt(252))

In [52]:
volatility = returns.std()*np.sqrt(252)

In [53]:
metrics = pd.DataFrame({
    'Volatility': volatility,
    'Cumulative Returns': cumulative_returns.iloc[-1],
    'Max Drawdown': max_drawdowns,
    'Sharpe Ratio': sharpe_ratio,
    'Sortino Ratio': sortino_ratio
})

print("Metrics for Indices and Equities:")
print(metrics)

Metrics for Indices and Equities:
        Volatility  Cumulative Returns  Max Drawdown  Sharpe Ratio  \
AAPL      0.281276           25.083426     -0.967691      0.870323   
AMZN      0.326308           14.750559     -0.970893      0.678203   
GOOGL     0.268456            5.843728     -0.927170      0.530993   
MSFT      0.257072           12.006318     -0.947725      0.727167   
TSLA      0.564307          102.166575     -0.997431      0.849101   
^GDAXI    0.200038            1.632538     -0.688274      0.305559   
^GSPC     0.174666            2.680068     -0.786810      0.435525   
^HSI      0.196581           -0.088379     -0.557008     -0.077339   
^IXIC     0.202644            4.296515     -0.869731      0.534083   
^N225     0.201934            1.708307     -0.733943      0.435742   

        Sortino Ratio  
AAPL         1.399279  
AMZN         1.264967  
GOOGL        0.814804  
MSFT         1.068513  
TSLA         2.738837  
^GDAXI       0.349380  
^GSPC        0.434823  
^HS